## Introduction

The idea is to have a twitter bot that allows users to tweet prompts @modernseinfeld_bot and receive a reply with the completion

The Twitter API requires that all requests use OAuth to authenticate. Once you sign up for twitter's developer platform https://developer.twitter.com/ you will get the required authentication credentials to be able to use the API. These credentials are three text strings:

- API key (token)
- API secret
- Bearer token

I store these in the .env file of the main project directory. This file is never checked into GitHub (by means of the .ignore file). We use load_dotenv() to load the required keys into our notebook.

In [2]:
import os
try:
    from dotenv import load_dotenv
except:
    !pip install python-dotenv
    from dotenv import load_dotenv
    
load_dotenv(override=True)
BEARER_TOKEN = os.getenv('TWITTER_BEARER_TOKEN')
CONSUMER_KEY = os.getenv('TWITTER_API_KEY')
CONSUMER_SECRET = os.getenv('TWITTER_API_SECRET')
ACCESS_TOKEN = os.getenv('TWITTER_ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
FINE_TUNED_MODEL = "curie:ft-personal-2022-02-16-02-07-10"

In [10]:
!curl -X GET -H "Authorization: Bearer {BEARER_TOKEN}" "https://api.twitter.com/2/tweets/20"

{"data":{"id":"20","text":"just setting up my twttr"}}

In [ ]:
!curl -X GET -H "Authorization: Bearer {BEARER_TOKEN}" "https://api.twitter.com/2/tweets/20?expansions=author_id"

In [4]:
try:
    import tweepy
except: 
    !pip -qq install tweepy=4.6.0
    import tweepy

#print(tweepy.__version__)

# Tweepy version 4.6.0 added support for APIv2 filtered streaming
# via StreamingClient where you can add a rule
# e.g. streaming_client.add_rules(tweepy.StreamRule("@SeinfeldPlotBot"))
# Note our development account only has access to use Twitter API v2

# To connect as the user
client_user = tweepy.Client(
    consumer_key=CONSUMER_KEY, consumer_secret=CONSUMER_SECRET,
    access_token=ACCESS_TOKEN, access_token_secret=ACCESS_TOKEN_SECRET
)
# To connect as the app
client_app = tweepy.Client(BEARER_TOKEN)

4.6.0


In [5]:
try: 
    import openai
except:
    !pip -qq install openai
    import openai
    
def call_GPT3(prompt: str) -> str:
    """\
    Prompt our fine-tuned GPT-3 model and return the response.
    
    Args:
        prompt (str): the prompt part of the answer. Must be less than 55 charachters.
    
    Returns:
        str: the prompt plus the completion from GPT3     
    """
    openai.api_key = OPENAI_KEY
    # We strip the completions of the stop token from training
    STOP_TOKEN = " ##END##"
    assert len(prompt) < 55

    #TODO Handle RateLimitError. For some reason the first API call after a long break will always raise this
    completion = openai.Completion.create(
        model=FINE_TUNED_MODEL,
        prompt=prompt,
        best_of = 8, #returns the "best" of 5 server-side completions (the one with the lowest log probability per token)
        n=1,
        max_tokens = 55)['choices'][0]
    
    return prompt + completion['text'].replace('\t','',-1).replace(str(STOP_TOKEN),"",-1)

In [6]:
def tweet_reply(text: str, tweet_id_to_reply: str) -> str:
    """\
    Reply to the mentioner with the text response.
    
    Args:
        text (str): the text of the tweet
        tweet_id_to_reply (str): the tweet to reply to
    
    Returns:
        str: the id of the reply tweet      
    """
    
    response = client_user.create_tweet(
        in_reply_to_tweet_id = tweet_id_to_reply,
        text=text
    )
    return f"https://twitter.com/user/status/{response.data['id']}"
    

In [85]:
# Manually poll to get mentions of SeinfeldPlotBot
user_id = client_app.get_user(username='SeinfeldPlotBot').data.id
print("The SeinfeldPlotBot user_id is {}".format(user_id))
response = client.get_users_mentions(user_id, tweet_fields=["author_id"])

The SeinfeldPlotBot user_id is 1494776018313392130


In [103]:
# Iterate through the polled responses and call GPT3
for tweet in response.data:
    print(tweet.data)
    prompt = tweet.data['text'].lstrip("@SeinfeldPlotBot")
    completion = call_GPT3(prompt)
    response = tweet_reply(completion, tweet.data['id'])
    print(f"Replied with {completion} in this tweet: {response}")

{'author_id': '1494776018313392130', 'id': '1496299981627707395', 'text': '@SeinfeldPlotBot test'}
Replied with  test auditions for America's Next Top Model. Kramer tries to "stunt," but his friend the limo driver leaks his identity. ## in this tweet: https://twitter.com/user/status/1496592947797794826


In [11]:
# Create a filtered stream

# Customize the on_tweet handler to call GPT3
class streaming_mentions(tweepy.StreamingClient):

    def on_tweet(self, tweet):
        print(tweet.data)
        prompt = tweet.data['text'].lstrip("@SeinfeldPlotBot")
        completion = call_GPT3(prompt)
        response = tweet_reply(completion, tweet.data['id'])
        print(f"Replied with {completion} in this tweet: {response}")

streaming_client = streaming_mentions(BEARER_TOKEN)
streaming_client.add_rules(tweepy.StreamRule("@SeinfeldPlotBot has:mention -is:retweet -is:reply -is:quote"))

#Launch the stream
streaming_client.filter()

{'id': '1498744649309118470', 'text': '@SeinfeldPlotBot Kramer goes to his reunion'}
Replied with  Kramer goes to his reunion dressed as a woman. "I didn't know how else to stand out!" in this tweet: https://twitter.com/user/status/1498744702463578112


Stream encountered an exception
Traceback (most recent call last):
  File "/Users/dan/dev/miniconda3/lib/python3.9/site-packages/tweepy/streaming.py", line 91, in _connect
    self.on_data(line)
  File "/Users/dan/dev/miniconda3/lib/python3.9/site-packages/tweepy/streaming.py", line 870, in on_data
    self.on_tweet(tweet)
  File "<ipython-input-11-3f8a3f241dfd>", line 9, in on_tweet
    completion = call_GPT3(prompt)
  File "<ipython-input-5-a443dad276c4>", line 20, in call_GPT3
    assert len(prompt) < 55
AssertionError


{'id': '1498744702463578112', 'text': 'Kramer goes to his reunion dressed as a woman. "I didn\'t know how else to stand out!"'}


### Findings
1. OpenAI tends to give you a RateLimit error the first time you call your model after a long time. Should be handled with a retry
2. Twitter streaming filter returns replies to mentions, even if you try to exclude this in the rule. This means that the bot's own response gets picked up as a new prompt. This should be handled in the code (since it can't be in the rule)
3. MOST IMPORTANTLY, OpenAI restricts this use case from going live. They have a mandatory go-live review if you're going to open up to more than 3 users. I submitted an application along with a video overview of my application. They got back to me in a couple of days and were very nice about it, but could not approve my twitterbot. According to their usage guidelines https://beta.openai.com/docs/usage-guidelines/twitter, introducing synthetic content into society is too risky. "All applications that involve social media must require a human to manually review and post the content. No automated or scheduled posting. Tweet generation and Instagram post generation are disallowed."
4. THUS, see the WebUI_FastAPI notebook. This is the UI I'm working on and will submit as my go-live application. 